In [1]:
import pandas as pd 
import numpy as np
import requests
import json
import time
import datetime as dt
import re
import os.path
from openpyxl import load_workbook
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
Area_Code = {
    '012':['Algeria','DZA'],
    '024':['Angola','AGO'],
    '204':['Benin','BEN'],
    '072':['Botswana','BWA'],
    '854':['Burkina Faso','BFA'],
    '108':['Burundi','BDI'],
    '132':['Cabo Verde','CPV'],
    '120':['Cameroon','CMR'],
    '140':['Central African Republic','CAF'],
    '148':['Chad','TCD'],
    '174':['Comoros','COM'],
    '178':['Congo','COG'],
    '384':['Côte d’Ivoire','CIV'],
    '180':['Democratic Republic of the Congo','COD'],
    '262':['Djibouti','DJI'],
    '818':['Egypt','EGY'],
    '226':['Equatorial Guinea','GNQ'],
    '232':['Eritrea','ERI'], #232
    '748':['Eswatini','SWZ'],
    '231':['Ethiopia','ETH'],
    '266':['Gabon','GAB'],
    '270':['Gambia','GMB'],
    '288':['Ghana','GHA'],
    '324':['Guinea','GIN'],
    '624':['Guinea-Bissau','GNB'],
    '404':['Kenya','KEN'],
    '426':['Lesotho','LSO'],
    '430':['Liberia','LBR'],
    '434':['Libya','LBY'],
    '450':['Madagascar','MDG'],
    '454':['Malawi','MWI'],
    '466':['Mali','MLI'],
    '478':['Mauritania','MRT'],
    '480':['Mauritius','MUS'],
    '504':['Morocco','MAR'],
    '508':['Mozambique','MOZ'],
    '516':['Namibia','NAM'],
    '562':['Niger','NER'],
    '566':['Nigeria','NGA'],
    '646':['Rwanda','RWA'],
    '686':['Senegal','SEN'],
    '690':['Seychelles','SYC'],
    '694':['Sierra Leone','SLE'],
    '706':['Somalia','SOM'],
    '710':['South Africa','ZAF'],
    '728':['South Sudan','SSD'],
    '678':['So Tom & Prncipe','STP'],
    '729':['Sudan','SDN'],
    '768':['Togo','TGO'],
    '788':['Tunisia ','TUN'],
    '800':['Uganda','UGA'],
    '834':['United Republic of Tanzania','TZA'], #835 834
    '894':['Zambia','ZMB'],
    '716':['Zimbabwe','ZWE'],
}

# Getting year for citation
D = dt.datetime.today()
y = D.year

In [ ]:
DataSource = 'United Nations Statistics Division'
DataProviderID= 9
Uncomtrade = pd.DataFrame(columns=['rt3ISO','rtTitle','DataSource','DataProviderId','SpecificSource','Indicatorcode','cmdDescE','Modecode','Unitcode','pfCode', 'yr', 'aggrLevel','rgCode', 'rgDesc', 'ptCode', 'ptTitle', 'pt3ISO', 'cmdCode','TradeValue', 'CIFValue', 'FOBValue'])
years = list(range(2015,2021))
for y in years:
    print('Data For ',y)
    for k in Area_Code.keys():
        
        print("\t",Area_Code[k][0],': Loading')   
        
        url = f'https://comtrade.un.org/api/get?r={k}&px=S2&ps={y}&p=All&cc=AG1&fmt=json'
        data = None
        Data = []
        a = 0
        while data is None:
            try:
                a +=1
                print("\t atempt:",a)
                data = requests.get(url).json()
                Data = data["dataset"]
            except:
                time.sleep(5)
                continue
        
        if Data != []: 
            DataF = pd.DataFrame(Data)
            DataF = DataF[['rt3ISO','rtTitle','cmdDescE','pfCode','yr','aggrLevel','rgCode','rgDesc','ptCode', 'ptTitle', 'pt3ISO','cmdCode','TradeValue','CIFValue', 'FOBValue']]
            
            M_code=[]
            for d in DataF['rgCode']:
                if d==1:
                    M_code.append('imp')
                else:
                    M_code.append('exp')
            DataF.insert(2,'Unitcode',['USD']*len(Data))        
            DataF.insert(2,'Modecode',M_code)
            Ind_code = 'S' + DataF['cmdCode']
            DataF.insert(2,'Indicatorcode',Ind_code)
            
            cite = f'Retrived from: {url}, On: {str(D.date())}, By web scraping using the python libraries: requests and pandas'
            SpecificSource = [cite]*len(Data)
            DataF.insert(2, 'SpecificSource', SpecificSource )
            DataF.insert(2, 'DataProviderId', [DataProviderID]*len(Data) )
            DataF.insert(2, 'DataSource', [DataSource]*len(Data) )
            
            Uncomtrade = pd.concat([Uncomtrade, DataF], ignore_index=True, sort=False)
            #print("\t",Area_Code[k][0],': Done')
        else:
            print("Data Not Found",)
        time.sleep(3)
        urll = f'https://comtrade.un.org/api/get?r={k}&px=S2&ps={int(y)}&p=All&cc=TOTAL&fmt=json'
        data1 = None
        Data1 = []
        a1 = 0
        while data1 is None:
            try:
                a1 +=1
                print("\t atempt:",a1)
                data1 = requests.get(urll).json()
                Data1 = data1["dataset"]
            except:
                time.sleep(5)
                continue
        
        if Data1 != []: 
            DataF1 = pd.DataFrame(Data1)
            DataF1 = DataF1[['rt3ISO','rtTitle','cmdDescE','pfCode','yr','aggrLevel','rgCode','rgDesc','ptCode', 'ptTitle', 'pt3ISO','cmdCode','TradeValue','CIFValue', 'FOBValue']]
            
            Ind_code1=[]
            for d1 in DataF1['rgCode']:
                if d1==1:
                    Ind_code1.append('XIMP')
                else:
                    Ind_code1.append('XEXP')
            DataF1.insert(2,'Unitcode',['USD']*len(Data1))        
            DataF1.insert(2,'Modecode',['TL']*len(Data1))
            #Ind_code1 = 'S' + DataF1['cmdCode']
            DataF1.insert(2,'Indicatorcode',Ind_code1)
            
            cite1 = f'Retrived from: {url}, On: {str(D.date())}, By web scraping using the python libraries: requests and pandas'
            SpecificSource1 = [cite1]*len(Data1)
            DataF1.insert(2, 'SpecificSource', SpecificSource1 )
            DataF1.insert(2, 'DataProviderId', [DataProviderID]*len(Data1) )
            DataF1.insert(2, 'DataSource', [DataSource]*len(Data1) )
            
            Uncomtrade = pd.concat([Uncomtrade, DataF1], ignore_index=True, sort=False)
            print("\t",Area_Code[k][0],': Done')
        else:
            print("Data Not Found",)
        
Uncomtrade.rename(columns = {'rt3ISO':'Areacode','rtTitle':'Areaenglish','cmdDescE':'Indicatorname',}, inplace = True)            
Uncomtrade.to_excel('UNComtrade.xlsx', index=False)
print('All Done!')

Data For  2015
	 Algeria : Loading
	 atempt: 1
	 atempt: 1
	 Algeria : Done
	 Angola : Loading
	 atempt: 1
	 atempt: 1
	 Angola : Done
	 Benin : Loading
	 atempt: 1
	 atempt: 1
	 Benin : Done
	 Botswana : Loading
	 atempt: 1
	 atempt: 1
	 Botswana : Done
	 Burkina Faso : Loading
	 atempt: 1
	 atempt: 1
	 Burkina Faso : Done
	 Burundi : Loading
	 atempt: 1
	 atempt: 1
	 Burundi : Done
	 Cabo Verde : Loading
	 atempt: 1
	 atempt: 1
	 Cabo Verde : Done
	 Cameroon : Loading
	 atempt: 1
	 atempt: 1
	 Cameroon : Done
	 Central African Republic : Loading
	 atempt: 1
	 atempt: 1
	 Central African Republic : Done
	 Chad : Loading
	 atempt: 1
Data Not Found
	 atempt: 1
Data Not Found
	 Comoros : Loading
	 atempt: 1
	 atempt: 1
	 Comoros : Done
	 Congo : Loading
	 atempt: 1
	 atempt: 1
	 Congo : Done
	 Côte d’Ivoire : Loading
	 atempt: 1
	 atempt: 1
	 Côte d’Ivoire : Done
	 Democratic Republic of the Congo : Loading
	 atempt: 1
	 atempt: 1
	 Democratic Republic of the Congo : Done
	 Djibouti : Lo

In [8]:
Uncomtrade[Uncomtrade['rtTitle']=='Cabo Verde']

,rt3ISO,rtTitle,DataSource,DataProviderId,SpecificSource,Indicatorcode,cmdDescE,Modecode,Unitcode,pfCode,...,aggrLevel,rgCode,rgDesc,ptCode,ptTitle,pt3ISO,cmdCode,TradeValue,CIFValue,FOBValue
3801,CPV,Cabo Verde,United Nations Statistics Division,9,Retrived from: https://comtrade.un.org/api/get...,S0,Food and live animals chiefly for food,imp,USD,S2,...,1,1,Import,0,World,WLD,0,163428824,None,None
3802,CPV,Cabo Verde,United Nations Statistics Division,9,Retrived from: https://comtrade.un.org/api/get...,S0,Food and live animals chiefly for food,exp,USD,S2,...,1,2,Export,0,World,WLD,0,57037481,None,None
3803,CPV,Cabo Verde,United Nations Statistics Division,9,Retrived from: https://comtrade.un.org/api/get...,S0,Food and live animals chiefly for food,imp,USD,S2,...,1,1,Import,4,Afghanistan,AFG,0,821,None,None
3804,CPV,Cabo Verde,United Nations Statistics Division,9,Retrived from: https://comtrade.un.org/api/get...,S0,Food and live animals chiefly for food,exp,USD,S2,...,1,2,Export,12,Algeria,DZA,0,543194,None,None
3805,CPV,Cabo Verde,United Nations Statistics Division,9,Retrived from: https://comtrade.un.org/api/get...,S0,Food and live animals chiefly for food,imp,USD,S2,...,1,1,Import,24,Angola,AGO,0,359,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4447,CPV,Cabo Verde,United Nations Statistics Division,9,Retrived from: https://comtrade.un.org/api/get...,XIMP,All Commodities,TL,USD,S2,...,0,1,Import,842,USA,USA,TOTAL,12306932,None,None
4448,CPV,Cabo Verde,United Nations Statistics Division,9,Retrived from: https://comtrade.un.org/api/get...,XEXP,All Commodities,TL,USD,S2,...,0,2,Export,842,USA,USA,TOTAL,1218534,None,None
4449,CPV,Cabo Verde,United Nations Statistics Division,9,Retrived from: https://comtrade.un.org/api/get...,XIMP,All Commodities,TL,USD,S2,...,0,1,Import,858,Uruguay,URY,TOTAL,802203,None,None
4450,CPV,Cabo Verde,United Nations Statistics Division,9,Retrived from: https://comtrade.un.org/api/get...,XIMP,All Commodities,TL,USD,S2,...,0,1,Import,860,Uzbekistan,UZB,TOTAL,9453,None,None
